## basic config

In [1]:
import torch
torch.cuda.device_count()

1

In [2]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/DragVideo/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [3]:
# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/home/bean/"

# keep matching encoder,SG as pairt 
old = {"encoder":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
       "SG": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
       "model_name":"stylegan2",
       "resolution":1024,
       }

new = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl",
         "model_name":"stylegan3",
               "resolution":1024,
         }

new_256 = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/Stylegan3_1_files/stylegan3-r-ffhqu-256x256.pkl",
         "model_name":"stylegan3",
         "resolution":256,
         }

videos = {
       "obama": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/obama.mp4",
       "man_speaking": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
       "rahul": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/rahul.mp4",
       "alien_girl": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/alien_girl.mp4",
       "mirrAR": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/mirrAR.mp4",
       "vsauce_frnd":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4",
       
}

In [4]:
EXPERIMENT_NAME_POSTFIX = "256_sg3_testing"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"
models_set = new_256
Testing =False
# --------------------------------  


import datetime
lazy_config = {
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+"_"+ EXPERIMENT_NAME_POSTFIX,
    # "EXP_NAME": "/home/bean/DragVideo/Data_store/experiments/_SAVE_mirrAR_200_frames_2",
    "e4e": models_set['encoder'],
    "stylegan2_ada_ffhq": models_set['SG'],
    "video_path":  videos["vsauce_frnd"],
    "model_name" : models_set['model_name'],
    "n_frames" : 50,#200,
    "IMAGE_SIZE": models_set["resolution"],
    "N_STEPS_in_draggan":  "100",
    "editing_function_name":"large_nose"#"large_eyes" # "make_jaw_wider" # "mouth_wide"
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI

hyper_config = {
    "max_pti_steps": 450,
    "first_inv_steps":200,
    "max_images_to_invert": 200,
}

if Testing: 
    hyper_config["max_pti_steps"] = 1
    hyper_config["first_inv_steps"] = 1
    hyper_config["max_images_to_invert"] = 5
    lazy_config["N_STEPS_in_draggan"] = '1'
    


# ----------------------------------------------------------------------------------------------

from run_utils import *

# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])



# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
     "quad_values_path": os.path.join(Experiment_path,'quad_values'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [5]:
# extract frames from video
#==============================================================================

from importlib.machinery import SourceFileLoader
# imports the module from the given path
video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")

video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])

 int(cam.get(cv2.CAP_PROP_FRAME_COUNT))= 327 ,n_frames=50 
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/000.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/001.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/002.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/003.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/004.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/005.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/006.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/007.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/raw/008.jpg
Creating.../ho

# start

In [6]:
# from utils.align_data import pre_process_images


In [7]:
# # # delete this cell 
# # "/home/bean/DragVideo/Data_store/man_speaking/aligned": input_data_path 
# # "/home/bean/DragVideo/Data_store/man_speaking/quad_values", quad_pkl_path

# raw_path = "/home/bean/DragVideo/Data_store/man_speaking/raw"
# pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path ,quad pickle dir


In [8]:
from utils.align_data import pre_process_images
pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path ,quad pickle dir
#
# align_faces_2(os.path.join(Experiment_path,'raw'))



100%|██████████| 50/50 [00:38<00:00,  1.30it/s]

saving aligned images...
saving quad values...
done!


In [9]:
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

generator_type =paths_config.multi_id_model_type if use_multi_id_training else "__"
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

# print(sg_tuned_pkl) # 'QBUXQCXZGWET'



#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)
print("pwd",os.getcwd())
import sys
sys.path.append(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
torch.Size([3, 256, 256]) 3 256 256
Setting up PyTorch plugin "bias_act_plugin"... Done.


  0%|          | 0/200 [00:00<?, ?it/s]

Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


...

torch.Size([3, 256, 256]) 3 256 256


100%|██████████| 200/200 [00:09<00:00, 21.14it/s]


torch.Size([3, 256, 256]) 3 256 256


100%|██████████| 450/450 [17:01<00:00,  2.27s/it]


Exporting large updated pickle based off new generator and ffhq.pkl
pwd /home/bean/DragVideo/DragGAN/_facial-landmarks-recognition
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/020.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/000.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/001.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/002.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/003.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/004.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/landmarks/005.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-22_

Runs draggan part

In [10]:

# # just to run this part seperately
# import os 
# Experiment_path = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_frnd_120_frames_wide_nose_with_border"
# sg_tuned_pkl = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_frnd_120_frames_wide_nose_with_border/tuned_SG/stylegan3_NXHOAZADQKTA.pkl"
# lazy_config["N_STEPS_in_draggan"]="160"
# lazy_config["editing_function_name"]="large_nose"
# lazy_config["IMAGE_SIZE"]="1024"


In [11]:
# clean all data from gpu
import torch
torch.cuda.empty_cache()




os.chdir(env_config["DragGAN_dir"])
import subprocess

print(f"sg_tuned_pkl: {sg_tuned_pkl}")

subprocess.call(['python', 'run_dragvideo.py',
                 '--Experiment_path', Experiment_path,
                 '--N_STEPS',       lazy_config["N_STEPS_in_draggan"],
                 '--CHECKPOINT_PATH', sg_tuned_pkl,
                 "--MAX_SIZE",      str(lazy_config["IMAGE_SIZE"]),
                 "--editing_function_name",lazy_config["editing_function_name"],
])
                #  "--verbose",       "False"])


sg_tuned_pkl: /home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/tuned_SG/stylegan3_DXJXUMNORDAY.pkl


editing_function_name: large_nose
intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/tuned_SG/stylegan3_DXJXUMNORDAY.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 32768, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9977843871238888, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 256, 'img_channels': 3, 'use_radial_filters': True}
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


  0%|          | 0/100 [00:00<?, ?it/s]/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 100/100 [00:05<00:00, 18.82it/s]


intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/tuned_SG/stylegan3_DXJXUMNORDAY.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 32768, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9977843871238888, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 256, 'img_channels': 3, 'use_radial_filters': True}


100%|██████████| 100/100 [00:05<00:00, 18.94it/s]


...

100%|██████████| 100/100 [00:05<00:00, 18.08it/s]


intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/tuned_SG/stylegan3_DXJXUMNORDAY.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 32768, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9977843871238888, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 256, 'img_channels': 3, 'use_radial_filters': True}


100%|██████████| 100/100 [00:05<00:00, 18.19it/s]


Done!


0

# post processing

In [12]:
import os
from tqdm import tqdm
os.getcwd()

'/home/bean/DragVideo/DragGAN'

In [13]:

from utils import de_alignment 
put_back_the_edited_image = de_alignment.put_back_the_edited_image
import importlib
importlib.reload(de_alignment)

<module 'utils.de_alignment' from '/home/bean/DragVideo/DragGAN/_PTI/utils/de_alignment.py'>

In [14]:
# raw_image_path = "/home/bean/DragVideo/Data_store/experiments/2023-08-18_03-45-37MAN_SPEAKING_QUAD_VALUES/raw/082.jpg"
# # aligned_image_path = "/home/bean/DragVideo/Data_store/experiments/2023-08-18_03-45-37MAN_SPEAKING_QUAD_VALUES/aligned/000.jpg"
# aligned_image_path2 = "/home/bean/DragVideo/Data_store/experiments/2023-08-18_03-45-37MAN_SPEAKING_QUAD_VALUES/aligned/082.jpg"
# quad = "/home/bean/DragVideo/Data_store/experiments/2023-08-18_03-45-37MAN_SPEAKING_QUAD_VALUES/quad_values/082.pkl"

In [15]:
# img = put_back_the_edited_image(raw_image_path,aligned_image_path2,quad,save_path='del.jpg',show=True)


In [16]:
# to paste_edited faces back for a folder of images:

# dir_path = "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames_SAVE_____________"


def paste_edited_faces_back(dir_path,edited_dir='after_drag',save_dir='edit_pasted'):
    raw_dir = os.path.join(dir_path,'raw')
    edited_dir = os.path.join(dir_path,edited_dir)
    quad_dir = os.path.join(dir_path,'quad_values')
    save_dir = os.path.join(dir_path,save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # get names from edited_dir
    names = [name.split('.')[0] for name in os.listdir(edited_dir)]
    for name in tqdm(names):
        raw_image = os.path.join(raw_dir,name+'.jpg')
        edited_image = os.path.join(edited_dir,name+'.png')
        quad_path = os.path.join(quad_dir,name+'.pkl')
        save_path = os.path.join(save_dir,name+'.png')
        put_back_the_edited_image(raw_image,edited_image,quad_path,save_path=save_path)


In [17]:
paste_edited_faces_back(Experiment_path,edited_dir="before_drag",save_dir="before_drag_pasted")

100%|██████████| 50/50 [00:07<00:00,  6.59it/s]


In [18]:
# dir_path = "/home/bean/DragVideo/Data_store/man_speaking"
paste_edited_faces_back(Experiment_path)

100%|██████████| 50/50 [00:07<00:00,  6.63it/s]


In [19]:

# temp_dir = os.path.join(Experiment_path,'before_drag_pasted')
# video_utils.make_video(temp_dir,video_name="before_pasted_back")

In [20]:


# from importlib.machinery import SourceFileLoader
# # imports the module from the given path
# video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()

In [21]:
# dir_path = "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames_SAVE_____________/edit_pasted"
# dir_path="/home/bean/DragVideo/Data_store/man_speaking/edit_pasted"

temp_dir = os.path.join(Experiment_path,'edit_pasted')
video_utils.make_video(temp_dir,video_name="pasted_back")

avi file deleted
avi file deleted


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1597178665428/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libsw

In [22]:
# video_utils.video_side_by_side("/home/bean/DragVideo/Data_store/OLD/original_videos/obama.mp4",
# '/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames_SAVE_____________/edit_pasted/pasted_back.mp4',
# '/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames_SAVE_____________/edit_pasted/obama_side_by_side')

In [23]:
v2 = os.path.join(Experiment_path,"edit_pasted","pasted_back.mp4")
v1 = lazy_config['video_path']
v3 = os.path.join(Experiment_path,"videos")#"/home/bean/DragVideo/Data_store/experiments/2023-08-20_00-49-31mirrAR_200_frames/videos"
video_utils.video_side_by_side(v1,v2,v3)

Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/000.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/001.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/002.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/003.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/004.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/005.jpg


Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/006.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/007.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/008.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/009.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/010.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/011.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/012.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/013.jpg
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-22_19-02-28_256_sg3_testing/videos/014.jpg
Creating.../home/bean/DragVideo/Data_store/exp

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1597178665428/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libsw

In [24]:
# video_path="/home/bean/DragVideo/Data_store/OLD/original_videos/man_results/man_124_edited.mp4" # need 124 frames 
# output_path="/home/bean/DragVideo/Data_store/man_speaking/after_drag"

In [25]:
# video_utils.extract_frames(video_path, output_path)# n_frames=1000, n_digits_in_name=5)

# extras


In [26]:
# path = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_frnd_120_frames_wide_nose/edit_pasted/000.png"

# from PIL import Image
# import numpy as np
# # open this image as it is. and check if it has a alpha channel
# img = Image.open(path)
# # img = img.convert("RGBA")
# img = np.array(img)
# print(img.shape)
